In [1]:
from RobotLink import *
from StereoCamera import *
from ParticleFilter import *
from probability_functions import *
from utils import *
import matplotlib.pyplot as plt
import time
# import ros
import rosbag

In [2]:
psm_arm = RobotLink('journal_dataset/LND.json')
cam = StereoCamera('journal_dataset/camera_calibration.yaml', rectify=True)

In [3]:
# Get a sample data!!
bag = rosbag.Bag('journal_dataset/stationary_camera_2020-06-24-15-49-10.bag')

ecm_pose_list  = []
ecm_pose_ts    = []
ecm_joint_list = []
ecm_joint_ts   = []
psm_joint_list = []
psm_joint_ts   = []
psm_jaw_list   = []
psm_jaw_ts     = []
left_img_list  = []
left_img_ts    = []
right_img_list = []
right_img_ts   = []

for topic, msg, t in bag.read_messages(topics=['/dvrk/ECM/position_cartesian_current', 
                                               '/dvrk/ECM/state_joint_current',
                                               '/dvrk/PSM1/state_joint_current',
                                               '/dvrk/PSM1/state_jaw_current',
                                               '/stereo/left/image',
                                               '/stereo/right/image']):
    if topic == '/dvrk/ECM/position_cartesian_current':
        ecm_pose_list.append(msg)
        ecm_pose_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/ECM/state_joint_current':
        ecm_joint_list.append(msg)
        ecm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_joint_current':
        psm_joint_list.append(msg)
        psm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_jaw_current':
        psm_jaw_list.append(msg)
        psm_jaw_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/left/image':
        left_img_list.append(msg)
        left_img_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/right/image':
        right_img_list.append(msg)
        right_img_ts.append(msg.header.stamp.to_sec())
    
#     if len(ecm_pose_list) == 1000:
#         break

bag.close()

ecm_pose_ts  = np.array(ecm_pose_ts)
ecm_joint_ts = np.array(ecm_joint_ts)
psm_joint_ts = np.array(psm_joint_ts)
psm_jaw_ts   = np.array(psm_jaw_ts)
left_img_ts  = np.array(left_img_ts)
right_img_ts = np.array(right_img_ts)

In [4]:
# Load hand-eye transform (this will be hard-coded for now according to the dataset)
f = open('journal_dataset/handeye.yaml')
hand_eye_data = yaml.load(f, Loader=yaml.FullLoader)

bc_T_cam = np.eye(4)
bc_T_cam[:-1, -1] = np.array(hand_eye_data['cam_tvec'])/1000.0
bc_T_cam[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['cam_rvec'])

cam_T_b = np.eye(4)
cam_T_b[:-1, -1] = np.array(hand_eye_data['PSM1_tvec'])/1000.0
cam_T_b[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['PSM1_rvec'])

In [5]:
# initialize filter
pf = ParticleFilter(num_states=9, 
                    initialDistributionFunc=sampleNormalDistribution,
                    #motionModelFunc=additiveGaussianNoise, \
                    motionModelFunc=lumpedErrorMotionModel,
                    obsModelFunc=pointFeatureObs,
                    num_particles=200)

initalize = True


# output video
left_video  = cv2.VideoWriter("left_video.mp4",  cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)
right_video = cv2.VideoWriter("right_video.mp4", cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)


for left_msg_idx, left_img_msg in enumerate(left_img_list):
    # Synchronize the data to the left_img_msg
    ts = left_img_ts[left_msg_idx]

    ecm_pose_msg  = ecm_pose_list[ np.argmin(np.abs(ecm_pose_ts - ts))]
    ecm_joint_msg = ecm_joint_list[np.argmin(np.abs(ecm_joint_ts - ts))]
    psm_joint_msg = psm_joint_list[np.argmin(np.abs(psm_joint_ts - ts))]
    psm_jaw_msg   = psm_jaw_list  [np.argmin(np.abs(psm_jaw_ts - ts))]
    # left_img_msg  = left_img_list[ left_msg_idx]
    right_img_msg = right_img_list[np.argmin(np.abs(right_img_ts - ts))]
    
    try:
        left_img  = np.ndarray(shape=(left_img_msg.height, left_img_msg.width, 3),
                                      dtype=np.uint8, buffer=left_img_msg.data)
        right_img = np.ndarray(shape=(right_img_msg.height, right_img_msg.width, 3),
                                      dtype=np.uint8, buffer=right_img_msg.data)
    except:
        continue

    left_img, right_img = cam.processImage(left_img, right_img)
    
    # Detect segmented keypoints and get new joint angle readings
    detected_keypoints_l, left_img  = segmentColorAndGetKeyPoints(left_img,  draw_contours=True)
    detected_keypoints_r, right_img = segmentColorAndGetKeyPoints(right_img, draw_contours=True)
    new_joint_angles = np.array(psm_joint_msg.position + psm_jaw_msg.position)

    start_t = time.time()
    psm_arm.updateJointAngles(new_joint_angles)
    
    if initalize:
        # Initialize particle filter
        initalize = False
        
        init_kwargs = {
                        "std": np.array([1.0e-3, 1.0e-3, 1.0e-3, # pos
                                         1.0e-2, 1.0e-2, 1.0e-2, # ori
                                         #5.0e-3, 5.0e-3, 0.02
                                         0.0, 0.0, 0.0])   # joints
                      }
        pf.initializeFilter(**init_kwargs)

    else:
        # Predict particle filter
        j_change = new_joint_angles - last_joint_angle_reading

        std_j = np.abs(j_change)*0.01
        std_j[-3:] = 0.0
        
        pred_kwargs = {
                        "std_pos": 2.5e-5, 
                        "std_ori": 1.0e-4,
                        "robot_arm": psm_arm, 
                        "std_j": std_j,
                        "nb": 4
                      }
        pf.predictionStep(**pred_kwargs)
    
    # Update particle filter
    upd_kwargs = {
                    "point_detections": (detected_keypoints_l, detected_keypoints_r), 
                    "robot_arm": psm_arm, 
                    "cam": cam, 
                    "cam_T_b": cam_T_b,
                    "joint_angle_readings": new_joint_angles,
                    "gamma": 0.15
    }
    
    pf.updateStep(**upd_kwargs)
    last_joint_angle_reading = new_joint_angles
    
    correction_estimation = pf.getMeanParticle()
    
    print("Time to update robot: {}ms".format((time.time()-start_t)*1000))
    
    # Project skeleton
    T = poseToMatrix(correction_estimation[:6])  
    new_joint_angles[-(correction_estimation.shape[0]-6):] += correction_estimation[6:]
    psm_arm.updateJointAngles(new_joint_angles)
    
    img_list = projectSkeleton(psm_arm.getSkeletonPoints(), np.dot(cam_T_b, T),
                               [left_img, right_img], cam.projectPoints)
    
    left_video.write( img_list[0])
    right_video.write(img_list[1])
    
left_video.release()
right_video.release()

/home/arclab/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1662: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/arclab/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/home/arclab/.local/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


Time to update robot: 165.118932724ms
Time to update robot: 185.173034668ms
Time to update robot: 167.328119278ms
Time to update robot: 156.162023544ms
Time to update robot: 204.675912857ms
Time to update robot: 209.167003632ms
Time to update robot: 161.274194717ms
Time to update robot: 157.993078232ms
Time to update robot: 154.869794846ms
Time to update robot: 155.17616272ms
Time to update robot: 241.796016693ms
Time to update robot: 151.765108109ms
Time to update robot: 226.159095764ms
Time to update robot: 151.964902878ms
Time to update robot: 210.269927979ms
Time to update robot: 206.248044968ms
Time to update robot: 277.934074402ms
Time to update robot: 205.872058868ms
Time to update robot: 202.862977982ms
Time to update robot: 187.868833542ms
Time to update robot: 153.578042984ms
Time to update robot: 178.506135941ms
Time to update robot: 181.487083435ms
Time to update robot: 205.796003342ms
Time to update robot: 183.333873749ms
Time to update robot: 181.288003922ms
Time to updat

Time to update robot: 169.220924377ms
Time to update robot: 129.992961884ms
Time to update robot: 134.024858475ms
Time to update robot: 136.303901672ms
Time to update robot: 143.442869186ms
Time to update robot: 181.15401268ms
Time to update robot: 93.729019165ms
Time to update robot: 149.007081985ms
Time to update robot: 96.3749885559ms
Time to update robot: 112.128019333ms
Time to update robot: 106.477975845ms
Time to update robot: 92.6921367645ms
Time to update robot: 93.8651561737ms
Time to update robot: 95.2389240265ms
Time to update robot: 95.8340167999ms
Time to update robot: 104.196071625ms
Time to update robot: 105.392932892ms
Time to update robot: 93.2748317719ms
Time to update robot: 93.8918590546ms
Time to update robot: 92.8139686584ms
Time to update robot: 95.7901477814ms
Time to update robot: 91.903924942ms
Time to update robot: 93.7809944153ms
Time to update robot: 135.755062103ms
Time to update robot: 93.3141708374ms
Time to update robot: 94.0430164337ms
Time to update 

Time to update robot: 231.279850006ms
Time to update robot: 162.978172302ms
Time to update robot: 163.418054581ms
Time to update robot: 246.343135834ms
Time to update robot: 234.720945358ms
Time to update robot: 236.054897308ms
Time to update robot: 206.018924713ms
Time to update robot: 125.928878784ms
Time to update robot: 163.719892502ms
Time to update robot: 175.19402504ms
Time to update robot: 122.485876083ms
Time to update robot: 160.813093185ms
Time to update robot: 120.055913925ms
Time to update robot: 172.027826309ms
Time to update robot: 139.053821564ms
Time to update robot: 140.750169754ms
Time to update robot: 119.752168655ms
Time to update robot: 118.062973022ms
Time to update robot: 159.821987152ms
Time to update robot: 159.579992294ms
Time to update robot: 157.850980759ms
Time to update robot: 163.491010666ms
Time to update robot: 134.567022324ms
Time to update robot: 101.464033127ms
Time to update robot: 141.802072525ms
Time to update robot: 126.489877701ms
Time to updat

Time to update robot: 97.4850654602ms
Time to update robot: 98.8299846649ms
Time to update robot: 97.2211360931ms
Time to update robot: 168.262004852ms
Time to update robot: 97.503900528ms
Time to update robot: 121.000051498ms
Time to update robot: 100.558996201ms
Time to update robot: 98.6268520355ms
Time to update robot: 94.260931015ms
Time to update robot: 96.4908599854ms
Time to update robot: 108.558893204ms
Time to update robot: 93.7390327454ms
Time to update robot: 129.343032837ms
Time to update robot: 128.922939301ms
Time to update robot: 96.8680381775ms
Time to update robot: 94.9029922485ms
Time to update robot: 102.396011353ms
Time to update robot: 94.7561264038ms
Time to update robot: 115.280151367ms
Time to update robot: 96.734046936ms
Time to update robot: 98.2179641724ms
Time to update robot: 97.4409580231ms
Time to update robot: 97.4988937378ms
Time to update robot: 98.1819629669ms
Time to update robot: 101.906061172ms
Time to update robot: 96.853017807ms
Time to update r

Time to update robot: 154.887914658ms
Time to update robot: 148.882865906ms
Time to update robot: 222.404003143ms
Time to update robot: 159.148931503ms
Time to update robot: 328.508138657ms
Time to update robot: 225.597143173ms
Time to update robot: 200.630903244ms
Time to update robot: 276.821136475ms
Time to update robot: 161.592006683ms
Time to update robot: 169.235944748ms
Time to update robot: 201.025009155ms
Time to update robot: 261.427879333ms
Time to update robot: 225.508928299ms
Time to update robot: 250.545978546ms
Time to update robot: 183.013916016ms
Time to update robot: 122.087955475ms
Time to update robot: 120.828866959ms
Time to update robot: 106.063127518ms
Time to update robot: 94.8989391327ms
Time to update robot: 102.996826172ms
Time to update robot: 95.4110622406ms
Time to update robot: 95.03698349ms
Time to update robot: 98.2890129089ms
Time to update robot: 111.603021622ms
Time to update robot: 93.8539505005ms
Time to update robot: 99.9970436096ms
Time to update

Time to update robot: 148.415088654ms
Time to update robot: 158.974170685ms
Time to update robot: 178.928136826ms
Time to update robot: 117.452859879ms
Time to update robot: 129.647016525ms
Time to update robot: 127.434015274ms
Time to update robot: 138.62991333ms
Time to update robot: 132.057905197ms
Time to update robot: 118.330001831ms
Time to update robot: 145.49612999ms
Time to update robot: 130.392074585ms
Time to update robot: 108.021020889ms
Time to update robot: 129.31394577ms
Time to update robot: 123.723983765ms
Time to update robot: 133.810997009ms
Time to update robot: 125.207185745ms
Time to update robot: 122.001886368ms
Time to update robot: 119.974851608ms
Time to update robot: 122.272014618ms
Time to update robot: 121.742963791ms
Time to update robot: 123.210906982ms
Time to update robot: 118.20602417ms
Time to update robot: 119.722127914ms
Time to update robot: 121.163129807ms
Time to update robot: 121.892929077ms
Time to update robot: 138.290166855ms
Time to update r

Time to update robot: 125.635147095ms
Time to update robot: 230.042934418ms
Time to update robot: 116.22595787ms
Time to update robot: 166.949033737ms
Time to update robot: 115.231990814ms
Time to update robot: 122.004985809ms
Time to update robot: 167.21701622ms
Time to update robot: 160.326004028ms
Time to update robot: 172.350168228ms
Time to update robot: 136.721849442ms
Time to update robot: 194.776058197ms
Time to update robot: 158.896923065ms
Time to update robot: 170.640945435ms
Time to update robot: 116.528987885ms
Time to update robot: 129.018068314ms
Time to update robot: 160.097122192ms
Time to update robot: 120.455980301ms
Time to update robot: 115.095853806ms
Time to update robot: 109.396934509ms
Time to update robot: 137.143850327ms
Time to update robot: 109.009027481ms
Time to update robot: 92.766046524ms
Time to update robot: 106.761932373ms
Time to update robot: 93.367099762ms
Time to update robot: 96.1511135101ms
Time to update robot: 115.2780056ms
Time to update rob

Time to update robot: 144.796133041ms
Time to update robot: 169.363021851ms
Time to update robot: 161.679029465ms
Time to update robot: 160.429000854ms
Time to update robot: 159.510135651ms
Time to update robot: 172.815084457ms
Time to update robot: 148.038864136ms
Time to update robot: 148.218154907ms
Time to update robot: 151.015996933ms
Time to update robot: 174.94392395ms
Time to update robot: 150.007009506ms
Time to update robot: 136.828899384ms
Time to update robot: 125.538825989ms
Time to update robot: 121.799945831ms
Time to update robot: 120.923042297ms
Time to update robot: 126.843929291ms
Time to update robot: 126.077890396ms
Time to update robot: 120.468854904ms
Time to update robot: 120.217084885ms
Time to update robot: 119.876146317ms
Time to update robot: 123.126983643ms
Time to update robot: 111.439943314ms
Time to update robot: 101.732969284ms
Time to update robot: 99.8079776764ms
Time to update robot: 101.482868195ms
Time to update robot: 99.6201038361ms
Time to updat

Time to update robot: 103.223085403ms
Time to update robot: 131.827116013ms
Time to update robot: 129.644155502ms
Time to update robot: 93.2679176331ms
Time to update robot: 140.403032303ms
Time to update robot: 92.2520160675ms
Time to update robot: 95.9188938141ms
Time to update robot: 110.181093216ms
Time to update robot: 94.269990921ms
Time to update robot: 108.60991478ms
Time to update robot: 94.7241783142ms
Time to update robot: 93.6748981476ms
Time to update robot: 98.8910198212ms
Time to update robot: 91.845035553ms
Time to update robot: 139.4739151ms
Time to update robot: 128.470897675ms
Time to update robot: 94.269990921ms
Time to update robot: 104.419946671ms
Time to update robot: 137.897014618ms
Time to update robot: 93.48487854ms
Time to update robot: 94.2540168762ms
Time to update robot: 111.862897873ms
Time to update robot: 94.6769714355ms
Time to update robot: 133.447170258ms
Time to update robot: 96.4570045471ms
Time to update robot: 110.54110527ms
Time to update robot: